<a href="https://colab.research.google.com/github/rajendradayma/NLP-LLM-Text-Summarization-Geo-Tagged-QA-Pipeline/blob/main/Text_Summarization_Pipeline_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*THIS FIRST APPROACH

In [2]:
!pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.5 MB/s eta 0:

In [10]:
# Install required packages
!pip install -q kaggle transformers datasets sentencepiece spacy geopy pandas tqdm
!python -m spdown spacy download en_core_web_lg -q

import os
import pandas as pd
from transformers import pipeline
import spacy
from geopy.geocoders import Nominatim
from tqdm import tqdm
import torch # Import the torch module

# 1. SET UP KAGGLE API (one-time setup)
# Create the kaggle.json file with your API credentials
import json
api_token = {"username":"rajendradayma","key":"6e3c5a35fe32480aef5fb51a659392d3"}  # Replace with your actual credentials

# Create .kaggle directory and write the credentials
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(api_token, f)
os.chmod('/root/.kaggle/kaggle.json', 600)

# 2. DOWNLOAD DATASET DIRECTLY
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail
!unzip -o newspaper-text-summarization-cnn-dailymail.zip

# 3. LOAD DATA
train_df = pd.read_csv('/content/cnn_dailymail/test.csv')
print(f"Dataset loaded with {len(train_df)} articles")
print(train_df.head(2))

# 4. INITIALIZE MODELS
summarizer = pipeline("summarization",
                     model="google/pegasus-cnn_dailymail",
                     device=0 if torch.cuda.is_available() else -1) # Now torch is defined and can be used
if not spacy.util.is_package("en_core_web_lg"):
    !python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")
geolocator = Nominatim(user_agent="geo_news_app")

# 5. PROCESSING FUNCTION
def process_article(text):
    # Summarization
    summary = summarizer(text, max_length=130, min_length=30)[0]['summary_text']

    # Location extraction
    doc = nlp(text)
    locations = list(set(ent.text for ent in doc.ents if ent.label_ in ("GPE", "LOC")))

    # Geocoding (with error handling)
    geo_data = {}
    for loc in locations:
        try:
            location = geolocator.geocode(loc)
            if location:
                geo_data[loc] = (location.latitude, location.longitude)
        except:
            continue

    return summary, locations, geo_data

# 6. PROCESS SAMPLE ARTICLES
results = []
sample_size = 5  # Adjust as needed

for i, row in tqdm(train_df.iloc[:sample_size].iterrows(), total=sample_size):
    summary, locations, geo_data = process_article(row['article'])
    results.append({
        'id': row['id'],
        'summary': summary,
        'locations': locations,
        'coordinates': geo_data
    })

# 7. SAVE RESULTS
output_df = pd.DataFrame(results)
output_df.to_csv('geo_summaries.csv', index=False)
print("Processing complete. Results saved to geo_summaries.csv")

/usr/bin/python3: No module named spdown
Dataset URL: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail
License(s): CC0-1.0
newspaper-text-summarization-cnn-dailymail.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  newspaper-text-summarization-cnn-dailymail.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  
Dataset loaded with 11490 articles
                                         id  \
0  92c514c913c0bdfe25341af9fd72b29db544099b   
1  2003841c7dc0e7c5b1a248f9cd536d727f27a45a   

                                             article  \
0  Ever noticed how plane seats appear to be gett...   
1  A drunk teenage boy had to be rescued by secur...   

                                          highlights  
0  Experts question if  packed out planes are put...  
1  Drunk teenage boy climbed into lion enclosure ...  


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


100%|██████████| 5/5 [00:25<00:00,  5.14s/it]

Processing complete. Results saved to geo_summaries.csv


# THIS SECOND APPROACH

In [4]:
!pip install transformers torch spacy geopy pandas tqdm
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 62.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import pandas as pd
import spacy
from tqdm import tqdm
from geopy.geocoders import Nominatim
from transformers import pipeline

# Load models
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
nlp = spacy.load("en_core_web_sm")
geolocator = Nominatim(user_agent="geo_pipeline")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

def summarize_text(text, min_words=50, max_words=800, max_token_length=1024):
    try:
        # Skip too-short texts
        if len(text.split()) < min_words:
            return "Text too short to summarize."

        # Truncate too-long texts
        words = text.split()
        if len(words) > max_words:
            text = " ".join(words[:max_words])

        # Tokenize input safely
        inputs = tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=max_token_length)
        inputs = inputs.to(model.device)

        # Generate summary
        summary_ids = model.generate(inputs, max_length=130, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

    except Exception as e:
        print(f"Summarization error: {e}")
        return "Summarization failed."

def extract_location(text):
    doc = nlp(text)
    return list(set([ent.text for ent in doc.ents if ent.label_ == "GPE"]))

def get_coordinates(location):
    try:
        loc = geolocator.geocode(location)
        if loc:
            return (loc.latitude, loc.longitude)
    except Exception as e:
        print(f"Geocoding error for location '{location}': {e}")
    return (None, None)

def answer_question(context, question="What is the article about?"):
    try:
        result = qa_pipeline(question=question, context=context)
        return result['answer']
    except Exception as e:
        print(f"QA error: {e}")
        return ""


In [7]:
df = pd.read_csv("/content/cnn_dailymail/test.csv")  # <-- Replace if filename is different
df = df.head(100)

In [8]:
tqdm.pandas()
results = []

In [9]:
for _, row in tqdm(df.iterrows(), total=len(df)):
    article = row.get("article", "")
    if not article: continue

    summary = summarize_text(article)
    locations = extract_location(article)
    lat, lon = (None, None)
    if locations:
        lat, lon = get_coordinates(locations[0])

    answer = answer_question(article)

    results.append({
        "original_text": article,
        "summary": summary,
        "qa_answer": answer,
        "location": locations[0] if locations else None,
        "latitude": lat,
        "longitude": lon
    })


100%|██████████| 100/100 [03:19<00:00,  1.99s/it]


In [12]:
output_df = pd.DataFrame(results)
output_df.to_csv("final_output.csv", index=False)
print("✅ Final output saved as final_output.csv")

✅ Final output saved as final_output.csv


In [14]:
from google.colab import files  # Import the 'files' object from google.colab

# ... your other code ...

files.download("final_output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>